In [1]:
'''
对于每一类diffusion扩散出来的视觉图像,构建non-shuffle的pt文件
'''

from torchvision import transforms # first, we need to preprocess(i.e.transform) the train/val sets
# then we need to load all the train / val sets
from torchvision.datasets import ImageFolder 
from torch.utils.data import DataLoader
import torch
from PIL import Image
from torch.utils.data import TensorDataset, DataLoader

rootDir = "/mnt/visual_sensory_moving2"

transforms = transforms.Compose([
    transforms.PILToTensor(),
    transforms.Resize((64,64)), # 裁剪为224*224
    ])

labelName = ["bowl", "dog", "feel", "get", "I", "know", "must", "sick", "you", "zero"]

labelBias = [128, 51, 72, 91, 114, 152, 129, 110, 145, 132]

labelCount = [2666, 2736, 2702, 2681, 2605, 2535, 2638, 2647, 2621, 2617]
labelCount = [i+400 for i in labelCount] # 最终的图片总数
# labelCount = [2794,2787,2774,2772,2719,2687,2767,2757,2766,2749]

label = 0
trainingRatio = 0.8
batchSize = 64
device = torch.device("cuda")

    
for i in range(len(labelName)):
    name = labelName[i] # bowl
    bias = labelBias[i]
    count = labelCount[i] # 2666+400=3066个
    tensorImgSets = []
    for j in range(count):
        img = Image.open(rootDir+"/%s/%s%04d.png" %(name, name, bias+j)) 
        tensorImg = transforms(img)
        tensorImg = tensorImg.reshape((1,-1)) # must be 1 * (3 * 224 * 224)
        if j == 0:
            tensorImgSets = tensorImg
        else:
            tensorImgSets = torch.concat((tensorImgSets, tensorImg), dim=0)
    
    imgLabels = torch.zeros((tensorImgSets.shape[0], 1), dtype=torch.uint8) # tensor.shape[0]:2666/2736/....
    imgLabels += label # 矩阵相加,label自动扩展为矩阵
    tensorImgSets = torch.concat((tensorImgSets, imgLabels), dim=-1)
    print(tensorImgSets.shape)
    
    label += 1
    
    torch.save(tensorImgSets,"/mnt/DCGAN_datasets/nonshuffle_diffusionDatasets_64_{}.pt".format(name))

/root/miniconda3/envs/myconda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([3066, 12289])
torch.Size([3136, 12289])
torch.Size([3102, 12289])
torch.Size([3081, 12289])
torch.Size([3005, 12289])
torch.Size([2935, 12289])
torch.Size([3038, 12289])
torch.Size([3047, 12289])
torch.Size([3021, 12289])
torch.Size([3017, 12289])
